In [ ]:
import arcpy
from arcpy.sa import *
import os

from utils.index_calculator import (
    calculate_ndvi,
    calculate_nirv,
    calculate_savi,
    calculate_cigreen,
    ensure_dir,
    calculate_statistics,
    export_statistics_to_csv
)

# --- Step 1: Setup ---
arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

input_raster = r"C:/your path/CathedralPines_2016.tif"
output_directory = r"C:/your path/SI"
ensure_dir(output_directory)

# Band mapping
blue = input_raster + r"\Band_1"
green = input_raster + r"\Band_2"
red = input_raster + r"\Band_3"
nir = input_raster + r"\Band_4"

# --- Step 2: Calculate Indices ---
indices = {
    "NDVI": calculate_ndvi(nir, red),
    "NIRv": calculate_nirv(nir, red),
    "SAVI": calculate_savi(nir, red),
    "CIgreen": calculate_cigreen(nir, green)
}

# --- Step 3: Save Rasters ---
for name, raster in indices.items():
    out_path = os.path.join(output_directory, f"{name}.tif")
    raster.save(out_path)
    print(f"{name} saved to {out_path}")

# --- Step 4: Compute and Export Stats ---
stats_dict = {}
for name in indices:
    raster_path = os.path.join(output_directory, f"{name}.tif")
    stats_dict[name] = calculate_statistics(raster_path)

csv_path = os.path.join(output_directory, "carbon_related_indices_stats.csv")
export_statistics_to_csv(stats_dict, csv_path)
print(f"Statistics saved to {csv_path}")

# --- Step 5: Cleanup ---
arcpy.CheckInExtension("Spatial")
